# Bangla Cyberbullying Classification with Random Forest

This notebook implements a random forest classifier for multi-class classification of Bangla cyberbullying text.


In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_curve,
    auc,
    precision_recall_curve,
)
from sklearn.preprocessing import label_binarize
from bnlp import SentencepieceTokenizer
from bnlp.embedding.fasttext import BengaliFasttext
from sklearn.preprocessing import LabelEncoder
import time
from itertools import cycle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings

warnings.filterwarnings("ignore")

# Set plot style
plt.style.use("ggplot")
sns.set(font_scale=1.2)
sns.set_style("whitegrid")

## 1. Loading and Exploring Datasets


In [2]:
# 1. Load the datasets
print("Loading datasets...")
original_cleaned_df = pd.read_csv("../../dataset/cleaned/original_cleaned.csv")
only_augmented_cleaned_df = pd.read_csv("../../dataset/cleaned/only_augmented.csv")

print(f"Original dataset shape: {original_cleaned_df.shape}")
print(f"Augmented dataset shape: {only_augmented_cleaned_df.shape}")

Loading datasets...
Original dataset shape: (41907, 6)
Augmented dataset shape: (6081, 3)


In [3]:
# Display the first few rows of original dataset
original_cleaned_df.head()

,comment,Category,Gender,comment react number,label,text_length
0,হালার পুত মদ খাওয়ার সময় রাতের বেলা মদ খাই দি...,Actor,Female,1.0,sexual,128
1,ঘরে শুট কেমন লেগেছে ক্যামেরাতে,Singer,Male,2.0,not bully,30
2,অরে বাবা টা পাগল,Actor,Female,2.0,not bully,16
3,ক্যাপ্টেন অফ বাংলাদেশ,Sports,Male,0.0,not bully,21
4,পটকা মাছ,Politician,Male,0.0,troll,8


## 2. Label Mapping and Visualization


In [4]:
# Map the labels for binary classification
# "not bully" = 0, all other categories = 1
def map_to_binary(label):
    if label == "not bully":
        return 0
    else:
        return 1

original_cleaned_df["label_encoded"] = original_cleaned_df["label"].apply(map_to_binary)
only_augmented_cleaned_df["label_encoded"] = only_augmented_cleaned_df["label"].apply(map_to_binary)

# Create a human-readable binary label for visualization
original_cleaned_df["binary_label"] = original_cleaned_df["label_encoded"].map({0: "not bully", 1: "bully"})
only_augmented_cleaned_df["binary_label"] = only_augmented_cleaned_df["label_encoded"].map({0: "not bully", 1: "bully"})

In [ ]:
# Check the distribution of binary labels
plt.figure(figsize=(10, 5))
sns.countplot(x="binary_label", data=original_cleaned_df, palette="viridis")
plt.title("Distribution of Binary Labels in Original Dataset")
plt.tight_layout()
plt.show()

## 3. Data Splitting and Augmentation


In [ ]:
# Split the original dataset into train and test sets
X = original_cleaned_df["comment"]
y = original_cleaned_df["label_encoded"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set size before augmentation: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

In [ ]:
# Add augmented data to the training set
X_aug = only_augmented_cleaned_df["text"]
y_aug = only_augmented_cleaned_df["label_encoded"]

# Concatenate original training data with augmented data
X_train_augmented = pd.concat([X_train, X_aug])
y_train_augmented = pd.concat([y_train, y_aug])

print(f"Training set size after augmentation: {len(X_train_augmented)}")

In [ ]:
# Visualize the class distribution before and after augmentation
plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
sns.countplot(x=y_train, palette="viridis")
plt.title("Class Distribution in Original Training Set")
plt.xlabel("Class Label")
plt.ylabel("Count")

plt.subplot(1, 2, 2)
sns.countplot(x=y_train_augmented, palette="viridis")
plt.title("Class Distribution After Augmentation")
plt.xlabel("Class Label")
plt.ylabel("Count")

plt.tight_layout()
plt.show()

## 4. Text Tokenization and Vectorization


In [ ]:
# Initialize SentencepieceTokenizer and BengaliFasttext
print("Initializing tokenizer and word embedding model...")
sp_tokenizer = SentencepieceTokenizer()
fasttext_model = BengaliFasttext()

In [ ]:
def tokenize_and_vectorize(text):
    # Tokenize the text
    tokens = sp_tokenizer.tokenize(text)

    # Get vector for each token and average them
    vectors = [
        fasttext_model.get_word_vector(token) for token in tokens if token.strip()
    ]

    # If no valid vectors, return zeros
    if not vectors:
        return np.zeros(300)  # FastText typically uses 300-dimensional vectors

    # Average the vectors
    return np.mean(vectors, axis=0)

In [11]:
# Apply tokenization and vectorization to training data
print("Vectorizing training data...")
start_time = time.time()
X_train_vectors = np.array([tokenize_and_vectorize(text) for text in X_train_augmented])
print(
    f"Training data vectorization completed in {time.time() - start_time:.2f} seconds"
)

In [4]:
# Apply tokenization and vectorization to testing data
print("Vectorizing testing data...")
start_time = time.time()
X_test_vectors = np.array([tokenize_and_vectorize(text) for text in X_test])
print(f"Testing data vectorization completed in {time.time() - start_time:.2f} seconds")

Vectorizing testing data...


NameError: name 'X_test' is not defined

In [ ]:
## 5. Training Random Forest Model


In [ ]:
# Configure and train Random Forest classifier
print("Training Random Forest model...")
# Parameters optimized for text embeddings
rf_model = RandomForestClassifier(
    n_estimators=200,  # More trees for better performance
    max_depth=None,  # Allow trees to grow fully
    min_samples_split=10,
    min_samples_leaf=4,
    max_features="sqrt",  # Common choice for high-dimensional data
    bootstrap=True,
    class_weight="balanced",  # Handle class imbalance
    random_state=42,
    n_jobs=-1,  # Use all processors
)

start_time = time.time()
rf_model.fit(X_train_vectors, y_train_augmented)
training_time = time.time() - start_time
print(f"Model trained in {training_time:.2f} seconds")

## 6. Model Evaluation - Basic Metrics


In [ ]:
# Make predictions and calculate basic metrics
print("Evaluating model...")
y_pred = rf_model.predict(X_test_vectors)

# Calculate basic metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
f1 = f1_score(y_test, y_pred, average="weighted")

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

In [ ]:
# Display classification report
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=["Not Bully", "Bully"]))

## 7. Confusion Matrix Visualization


In [ ]:
# Create and display confusion matrix
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred)
cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]  # Normalize
sns.heatmap(
    cm_norm,
    annot=True,
    fmt=".2f",
    cmap="Blues",
    xticklabels=["Not Bully", "Bully"],
    yticklabels=["Not Bully", "Bully"],
)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Normalized Confusion Matrix")
plt.tight_layout()
plt.show()

## 8. Feature Importance Analysis


In [ ]:
# Analyze feature importance
feature_importance = pd.DataFrame(
    {
        "Feature": [f"Feature_{i}" for i in range(X_train_vectors.shape[1])],
        "Importance": rf_model.feature_importances_,
    }
)
feature_importance = feature_importance.sort_values("Importance", ascending=False).head(
    20
)

plt.figure(figsize=(12, 8))
sns.barplot(x="Importance", y="Feature", data=feature_importance, palette="viridis")
plt.title("Top 20 Important Features")
plt.tight_layout()
plt.show()

## 9. ROC Curve Analysis


In [ ]:
# ROC Curve and AUC
y_score = rf_model.predict_proba(X_test_vectors)[:, 1]  # Probability of being "bully"

# Calculate ROC curve and ROC area
fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.tight_layout()
plt.show()

## 10. Precision-Recall Curve Analysis


In [ ]:
# Calculate Precision-Recall curve
precision, recall, _ = precision_recall_curve(y_test, y_score)
avg_precision = precision_score(y_test, y_pred)

plt.figure(figsize=(10, 8))
plt.plot(recall, precision, color='blue', lw=2, 
         label=f'Precision-Recall curve (AP = {avg_precision:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")
plt.tight_layout()
plt.show()

## 12. Model Summary and Saving


In [ ]:
# Summary of the model
print("Model Summary:")
print(f"Total samples in training set (with augmentation): {len(X_train_augmented)}")
print(f"Total samples in test set: {len(X_test)}")
print(f"Number of features (vector dimension): {X_train_vectors.shape[1]}")
print(f"Final model accuracy on test set: {accuracy:.4f}")
print(f"Training time: {training_time:.2f} seconds")

In [ ]:
# # Save the model if needed
# import joblib
print(f"Total samples in training set (with augmentation): {len(X_train_augmented)}")
print(f"Total samples in test set: {len(X_test)}")
print(f"Number of features (vector dimension): {X_train_vectors.shape[1]}")
print(f"Final model accuracy on test set: {accuracy:.4f}")
print(f"Training time: {training_time:.2f} seconds")